## UNO!

#### Set-Up Code:

In [ ]:
import random # For shuffling and randomly dealing cards
import time # For waiting so game doesn't run too quickly
import pickle # For saving game state

In [ ]:
# Defining card features for later use, encapsulating so aren't edited
_colours = ['red', 'blue', 'green', 'yellow']
_values = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
_actions = ['skip', 'reverse', 'draw two', 'swap hands']

# Main card class with colour and value attributes, method to show card
class Card():
    
    def __init__(self, colour, value):
        self._value = value
        self._colour = colour
            
    def show(self):
        if self._colour == None:
            return str(self._value)
        else:
            return "{}, {}".format(self._colour, self._value)

In [ ]:
# Normal card class inherits from card class - for numbered cards
class NormalCard(Card):
    
    def __init__(self, colour, value):
        super().__init__(colour, value)
    
    @property
    def get_colour(self):
        return self._colour
    
    @property
    def get_value(self):
        return self._value # Has encapsulated colour and value properties

In [ ]:
# Action card inherits from card class - for special card with functions
class ActionCard(Card):
    
    def __init__(self, colour, value):
        super().__init__(colour, value)
        
    @property
    def get_colour(self):
        return self._colour
    
    @property
    def get_value(self):
        return self._value # Has encapsulated colour and value properties

    # Methods for completing card actions: skip, reverse, draw 2 and swap hands
    # Next player misses turn
    def play_skip(self, game):
        game.next_player()
        print("")
        print("{} has been skipped!".format(game.players[game.current_player]))
        
    # Game continues in opposite direction
    def play_reverse(self, game):
        game.play_direction = game.play_direction * -1
        print("")
        print("Direction of play has been reversed!")
    
    # Next player recieves 2 cards
    def play_draw_two(self, game):
        game.next_player()
        for x in range(2):
            game.playersHands[game.players[game.current_player]].append(deck.draw())
        print("")
        print("{} has been given 2 cards!".format(game.players[game.current_player]))
            
    # Player can exchange cards with another (if don't want to swap can type own name)
    def play_swap_hands(self, game):
        while True:
            swap_player = input("Which player would you like to swap hands with? ").lower()
            if swap_player in game.players:
                game.playersHands[game.players[game.current_player]], game.playersHands[swap_player] = game.playersHands[swap_player], game.playersHands[game.players[game.current_player]]
                print("")
                print("{} has swapped hands with {}".format(game.players[game.current_player], swap_player))
                break
            else:
                print(f"Invalid player, choose from {game.players}")

In [ ]:
# Colour change class inherits from card class
class ColourChangeCard(Card):
    
    def __init__(self, value):
        super().__init__(None, value)
        
    @property
    def get_colour(self):
        return None
    
    @property
    def get_value(self):
        return self._value # Has encapsulated colour and value properties
    
    # Method for playing change colour (wild) card
    # Player decides what colour to change to (can be same as already is if don't want to swap)
    def play(self, game):
        while True:
            colour = input("Which colour would you like to change to? ")
            if colour.lower() in _colours:
                game.top_card_colour = colour.lower()
                print("")
                print(f"Colour changed to {game.top_card_colour}")
                break
            else:
                print(f"Invalid colour, choose from {_colours}")

In [ ]:
# Colour change draw 4 class inherits from card class
class ColourChangeDrawFourCard(Card):
    
    def __init__(self, value):
        super().__init__(None, value)
        
    @property
    def get_colour(self):
        return None
    
    @property
    def get_value(self):
        return self._value # Has encapsulated colour and value properties
    
    # Method for playing colour change draw 4 card
    # Player decides what colour to change to (can be same if wanted), and next player recieves 4 cards
    def play(self, game):
        while True:
            colour = input("Which colour would you like to change to? ")
            if colour.lower() in _colours:
                game.top_card_colour = colour.lower()
                print("")
                print(f"Colour changed to {game.top_card_colour}")
                game.next_player()
                for x in range(4):
                    game.playersHands[game.players[game.current_player]].append(deck.draw())
                print("{} has been given 4 cards!".format(game.players[game.current_player]))
                break
            else:
                print(f"Invalid colour, choose from {_colours}")

In [ ]:
# Deck class has methods to build and shuffle the deck, draw card from deck and flip discard pile when deck runs out
class Deck():
    
    def __init__(self):
        self.deck = []
        self._discard_pile = [] # Encapsulated so can't be edited
        self.build()
        
    def build(self):
        for colour in _colours:
            for value in _values:
                norm_card_format = NormalCard(colour, value)
                if value == 0:
                    self.deck.append(norm_card_format) # One 0 for each colour
                else:
                    self.deck.append(norm_card_format)
                    self.deck.append(norm_card_format) # Two of each other number for each colour
            for action in _actions:
                action_card_format = ActionCard(colour, action)
                self.deck.append(action_card_format)
                self.deck.append(action_card_format) # Two of each action card for each colour
        for i in range(4):
            self.deck.append(ColourChangeCard('colour change'))
            self.deck.append(ColourChangeDrawFourCard('colour change_draw four')) # Four colour change and colour chanege draw 4 cards
        self.shuffle()
    
    def shuffle(self):
        random.shuffle(self.deck)
    
    def draw(self):
        if len(self.deck) == 0:
            self.flip_pile()
        return self.deck.pop() # Takes card and removes from deck
    
    def flip_pile(self):
        top_card = self._discard_pile.pop()
        self.deck = self._discard_pile
        self.shuffle()
        self._discard_pile = [top_card] # Keep top card but flip rest of pile to become deck

In [ ]:
# Player class has methods to deal cards, for players hand to be shown and player to play a card
class Player():
    
    def __init__(self, name):
        self.name = name
        self.hand = []
        
    def deal(self, deck):
        for i in range(7):
            card = deck.draw()
            self.hand.append(card)
    
    def showhand(self):
        print("{}'s hand: {}".format(self.name, [card.show() for card in self.hand]))     
        
    def playcard(self, index):
        return self.hand.pop(index)

In [ ]:
# Game class has player setup, deal initial, current player hand, playable, check any moves, next player, call uno, save, load, play saved, and play methods 
class Game():
    
    def __init__(self):
        self.players = []
        self.players_ages = {}
        self.playersHands = {}
        self.current_player = 0
        self.play_direction = 1
        self.end_game = False
        self.top_card_colour = None # Sets details to how they should be at start of the game
    
    # Asks player to input how many players, their names and ages
    def player_setup(self):
        while True:
            try:
                num_players = int(input("How many people (2-4) are playing? "))
                if 2 <= num_players <= 4:
                    break
                else:
                    print("Enter integer in range 2-4")
            except ValueError:
                print("Enter integer in range 2-4")

        for player in range(num_players):
            while True:
                name = input(f"Enter name of player {player+1}: ")
                if name not in self.players:
                    self.players.append(name.lower())
                    self.playersHands[name.lower()] = []
                    break
                else:
                    print(f"{name.lower()} has already been entered")
                    
        for player in self.players:
            while True:
                try:
                    age = int(input(f"Enter age of player {player}: "))
                    self.players_ages[player] = age
                    break
                except ValueError:
                    print("Enter age integer")

    # Deals 7 cards to each player and adds to dictionary with player name
    def deal_initial(self):    
        for player in self.players:
            player_obj = Player(player)
            player_obj.deal(deck)
            self.playersHands[player] = player_obj.hand

    # Returns the cards in the current players hand
    def current_player_hand(self):
        player_name, player_hand = list(self.playersHands.keys()), list(self.playersHands.values())
        print(f"{player_name[self.current_player]}'s hand: ")
        for i, card in enumerate(player_hand[self.current_player], 1):
            print(f"{i}. {card.show()}")
        
    # Checks whether a card can be played
    def playable(self, card):
        top_card = deck._discard_pile[-1]
        if card.get_value == "colour change" or card.get_value == "colour change_draw four":
            return True # Can always be played
        elif top_card.get_colour == card.get_colour or top_card.get_value == card.get_value:
            return True # Checks if colour or value or same as top of discard pile
        elif top_card.get_value == "colour change" and self.top_card_colour == card.get_colour:
            return True
        elif top_card.get_value == "colour change_draw four" and self.top_card_colour == card.get_colour:
            return True # Comparing cards colour against colour that has been set
        else:
            return False
            
    # Check whether there are any playable cards in a hand
    def check_any_moves(self, player_name):
        player_hand = self.playersHands[player_name]
        for card in player_hand:
            if self.playable(card):
                return True
        return False
            
    # Move to next players go
    def next_player(self):
        if self.play_direction == 1:
            self.current_player = (self.current_player + 1) % len(self.players) # Ensuring can't go over no. players and reverse doesn't break
        else:
            self.current_player = (self.current_player - 1) % len(self.players)
        return self.current_player
        
    # Ask player if they have uno, deal a card to them if incorrect
    def call_uno(self):
        self.play_direction
        current_player = self.players[self.current_player]
        if len(self.playersHands[current_player]) != 0:
            while True:
                ask_uno = input("Do you have UNO? (Yes/No): ")
                if ask_uno.capitalize() == "Yes":
                    if len(self.playersHands[current_player]) == 1:
                        print("UNO!")
                        last_card = self.playersHands[current_player]
                        if last_card[0].get_value == "skip" or last_card[0].get_value == "reverse" or last_card[0].get_value == "draw two" or last_card[0].get_value == "swap hands" or last_card[0].get_value == "colour change" or last_card[0].get_value == "colour change_draw four":
                            print("You can't finish on a picture card, you will recieve 1 card!")
                            self.playersHands[current_player].append(deck.draw()) # Dealing card if last card if picture card, can only finish only normal/number card (house rule)
                    else:
                        print("Incorrect, you don't have UNO!")
                        print("You will recieve 1 card")
                        self.playersHands[current_player].append(deck.draw())
                    break
                elif ask_uno.capitalize() == "No":
                    if len(self.playersHands[current_player]) == 1:
                        print("Incorrect, you did have UNO!")
                        print("You will recieve 1 card")
                        self.playersHands[current_player].append(deck.draw())
                    else:
                        print("Correct, you don't have UNO!")
                    break
                else:
                    print("Enter yes or no")
                    
    # Save game in state currently in
    def save(self, game):
        with open('Saved Game.pickle', 'wb') as file:
            pickle.dump(game, file)
            
    # Load previously saved game
    def load(self, filename):
        try:
            with open('Saved Game.pickle', 'rb') as file:
                game = pickle.load(file)
            return game
        except FileNotFoundError:
            print("There is no saved game called this")
            return None
    
    # Setup for continuing playing previously saved game
    def play_saved(self):
        saved = self.load('Saved Game.pickle')
        if saved is not None:
            self.players = saved.players
            self.playersHands = saved.playersHands
            self.current_player = saved.current_player
            self.play_direction = saved.play_direction
            self.end_game = saved.end_game
            self.top_card_colour = saved.top_card_colour
            self.players_ages = saved.players_ages # Sets details to how left when play paused
        
    # Main play loop
    def play(self):
        youngest = min(self.players_ages, key=self.players_ages.get)
        self.current_player = self.players.index(youngest) # Players play in age order starting with youngest (house rule)
        while True:
            first_card = deck.draw() # Drawing start card, can't be colour change or colour change  draw 4, actions not used from action card starts
            if first_card.get_value != "colour change" and first_card.get_value != "colour change_draw four":
                deck._discard_pile.append(first_card)
                break
        while self.end_game == False:
            print("")
            print("{}'s turn".format(self.players[self.current_player]))
            time.sleep(0.5)
            if deck._discard_pile[-1].get_value == "colour change" or deck._discard_pile[-1].get_value == "colour change_draw four":
                print(f"Colour has been changed to: {self.top_card_colour}")
            else:
                print("Card on top of pile: {}".format(deck._discard_pile[-1].show())) # Showing player which card they need to match
            game.current_player_hand()
            current_player = self.players[self.current_player]
            if self.check_any_moves(current_player) == False:
                time.sleep(1)
                print("No valid cards, you will recieve a random number of cards between 1 and 5!")
                num_choice = random.randint(1,5)
                for x in range(num_choice):
                    self.playersHands[current_player].append(deck.draw())
                print(f"{current_player} has recieved {num_choice} cards!") # If no playable cards, player draws randomly allocated number of cards from 1-5 (house rule)
                if self.check_any_moves(current_player) == False:
                    time.sleep(1)
                    game.next_player()
                else:
                    print("You have picked up a valid card!")
                    print("You may take your turn again:") # Player can play a playable card straight after picking up if previously had no valid
            else:                        
                while True:
                    try:
                        choice = int(input("Choose index of card you would like to play from hand or 0 to save state and quit game: "))
                    except ValueError:
                        print("Enter the integer index of the card")
                    if choice == 0:
                        game.save(game)
                        print("Game paused and saved")
                        self.end_game = True
                        break
                    elif 1 <= choice <= len(self.playersHands[current_player]):
                        chosen_card = self.playersHands[current_player][choice-1]
                        if self.playable(chosen_card) == True:
                            card = self.playersHands[current_player].pop(choice - 1)
                            deck._discard_pile.append(card) # Check whether card exists and is playable
                            print("You played: {}".format(card.show()))
                            game.call_uno()
                            if card.get_value == "reverse":
                                action_card = ActionCard(card.get_colour, card.get_value)
                                action_card.play_reverse(self)
                            if card.get_value == "skip":
                                action_card = ActionCard(card.get_colour, card.get_value)
                                action_card.play_skip(self)
                            if card.get_value == "draw two":
                                action_card = ActionCard(card.get_colour, card.get_value)
                                action_card.play_draw_two(self)
                            if card.get_value == "colour change":
                                colour_change_card = ColourChangeCard(card.get_value)
                                colour_change_card.play(self)
                            if  card.get_value == "colour change_draw four":
                                colour_change_draw_card = ColourChangeDrawFourCard(card.get_value)
                                colour_change_draw_card.play(self)
                            if card.get_value == "swap hands":
                                action_card = ActionCard(card.get_colour, card.get_value)
                                action_card.play_swap_hands(self) # Play different functions if action card played
                            game.next_player()
                            break
                        else:
                            print("Not valid card")
                    else:
                        print("Not valid card index, choose within range") # Ask to re-try if card not valid
            if len(self.playersHands[current_player]) == 0:
                print("")
                print("{} is the winner! Game over".format(current_player))
                self.end_game = True # End game and print winner when player runs out of cards

#### Game Code:

In [ ]:
# Run to play game from start
deck = Deck()
deck.shuffle()
game = Game()
game.player_setup()
game.deal_initial()
game.play()

In [ ]:
# Run to play previously saved game
deck = Deck()
deck.shuffle()
game = Game()
game.play_saved()
game.play()